# Example: Recursive Implementation of Fibonacci Sequence Calculation
Recursion is a programming technique in which a function calls itself with a modified version of its input. This allows the function to repeat a process on a smaller scale, and the results of these smaller-scale processes can be combined to solve the original problem.

We illustrate recursion concepts by looking at different implementations of the computation of the `fibonacci` sequence in the `Compute.jl` file. 

* First, let's use the [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) to compute the average time required to compute the sequence $F_{0},\dots,F_{n}$ using the `vanilla` implementation of the `fibonacci` function (`for` loop based implementation that we explored previously). 
* Next, we'll benchmark a recursive implementation of the `fibonacci` function. The `fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that computes sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument. 
* Lastly, we'll benchmark a recursive function that uses memoization. The `memoization_fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that uses memoization to speed up the computation of the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument.

## Setup
This example uses functions encoded in the `src` directory and external third-party packages. In the `Include.jl` file, we load these functions to access them, set some required paths for this example and load external packages.

In [3]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-5/L5a`
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-5/L5a/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [5ae59095] + Colors v0.12.11
  [a93c6f00] + DataFrames v1.6.1
  [91a5bcdd] + Plots v1.40.8
  [10745b16] ~ Statistics ⇒ v1.10.0
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Examples-Fall-2024/lecture/week-5/L5a/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [d1d4a3ce] + BitFlags v0.1.9
  [944b1d66] + CodecZlib v0.7.6
  [35d6a980] + ColorSchemes v3.26.0
  [3da002f7] + ColorTypes v0.11.5
  [c3611d14] + ColorVectorSpace v0.10.0
  [5ae59095] + Colors v0.12.11
  [34da2185] + Compat v4.16.0
  [f0e56b4a] + ConcurrentUtilities v2.4.2
  [d38c429a] + Contour v0.6.3
  [a8cc5b0e] + Crayons v4.1.1
  [9a962f9c] + DataAPI v1.16.0
  [a93c6f00] + DataFrames v1.6.1
  [864edb3b] + DataStructures v0.18.20
  [e2d170a0] + DataValueInterfaces v1.0.0
  [8bb14

Let's set the number of elements of the Fibonacci sequence $F_{0},\dots,F_{n}$ that we want to compute in the `n` variable:

In [5]:
n = 25; # compute the Fibonacci sequence from F0 to F25

## Case 1: Test the basal `for` loop implementation of `fibonacci` sequence
Let's use the [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) to compute the average time required to compute the sequence $F_{0},\dots,F_{n}$ using the `vanilla` implementation of the `fibonacci` function (`for` loop based implementation that we explored previously). 

* The [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) exports the [@benchmarkable macro](https://juliaci.github.io/BenchmarkTools.jl/stable/reference/#BenchmarkTools.@benchmarkable-Tuple) which computes the runtime and memory profile of a function. It runs the function many times, and returns the statistical information about the performance.

In [7]:
test_run_basal = @benchmarkable fibonacci_for_loop_dict($(n));
tune!(test_run_basal)
result_basal = run(test_run_basal)

BenchmarkTools.Trial: 10000 samples with 125 evaluations.
 Range (min … max):  742.000 ns …  1.088 ms  ┊ GC (min … max):  0.00% … 99.90%
 Time  (median):     793.336 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   962.469 ns ± 10.900 μs  ┊ GC (mean ± σ):  15.44% ±  5.31%

   ██▅▁                                                         
  ▄█████▆▅▄▅▅▅▆▆▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  742 ns          Histogram: frequency by time         1.11 μs <

 Memory estimate: 1.78 KiB, allocs estimate: 7.

In [8]:
result_basal.times;

## Case 2: Test the `recursive` implementation of the `fibonacci` sequence
Let's benchmark a recursive implementation of the `fibonacci` function. The `fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that computes sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument. 
* __Q__: Recursion is cool. How does it perform relative to the `baseline` implementation of the `fibonacci` sequence calculation?

In [10]:
result_dictionary = Dict{Int,Int}()
test_run_recursive = @benchmarkable fibonacci!($(n), $(result_dictionary))
tune!(test_run_recursive)
result_recursive = run(test_run_recursive)

BenchmarkTools.Trial: 6642 samples with 1 evaluation.
 Range (min … max):  738.667 μs …  1.101 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     741.750 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   751.624 μs ± 24.401 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▃▃▂▂▂▁▂▁▁▁▂▁▁▁▁                                            ▁
  ████████████████████▇███▇█▇██▇▇▇▇▇▇▇▆▇▆▆▇▆▅▅▆▆▆▅▅▆▄▅▅▅▆▅▃▅▄▆ █
  739 μs        Histogram: log(frequency) by time       848 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
result_recursive.times;

## Case 3: Test the `recursive` implementation of `fibonacci` sequence with memoization
Finally, benchmark a recursive function that uses memoization. The `memoization_fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that uses memoization to speed up the computation of the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument.
* __Q__: Does the inclusion of the memoization change the runtime (or memory allocation) profile of the recursive implementation of the `fibonacci` sequence

In [13]:
result_dictionary_memo = Dict{Int,Int}()
test_run_recursive_memo = @benchmarkable memoization_fibonacci!($(n), $(result_dictionary_memo))
tune!(test_run_recursive_memo)
result_recursive_memo = run(test_run_recursive_memo)

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  7.966 ns … 44.377 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.091 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.107 ns ±  0.488 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                          █▁                                  
  ▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂ ▂
  7.97 ns        Histogram: frequency by time        8.26 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
result_recursive_memo.times;